Graph generation functions

In [1]:
using LightGraphs

invLogit(x) = 1./(1.+e.^-x)   

function addNode2(graph, p)
    add_vertex!(graph)
    x = nv(graph)
    degree = 0
    while degree ==0
        flips = rand(x-1)
        for i = 1:x-1
            if p[i]>flips[i]
                add_edge!(graph,i,x)
                degree +=1 
            end
        end
    end
    return graph
end

function addPrefNode(g,b,a_0 = -7)
    n = nv(g)
    L = laplacian_matrix(g)
    a = lufact(L) \ (b - mean(b))    
    p = invLogit(a+a_0)
    addNode2(g,p)
    push!(b,0)
    return g
end

function randEdgeGen(graph, newedges)
    for i in 1:newedges
        z = newedges
        x = collect(1:nv(graph))
        edge1 = rand(x)
        deleteat!(x, edge1)
        edge2 = rand(x)
        add_edge!(graph,edge1,edge2)
    end
    return graph
end

const MAX_ITER = 5000
const STOP_DIFF = 0.0001;

Optimization functions

In [2]:
# a update(Newton Raphson)


#y-1./(1.+e.^-(a+a_0))+(u' * L)' + rho*L*(L*a-b)   


function gradient2(a,a_0,u,L,rho,b,y)
#    grad = grad+ (y[i]-invLogit(a+a_0))+(u' * L)[0:t_0] + rho*(L*a-append!(b, zeros(t-t_o,1)))
    grad = y-invLogit(a+a_0)+(u' * L)' - rho*L*(L*a-b)
    return grad
end;



function hessian(a,a_0,rho,L)
    hess = Diagonal(vec((invLogit(a+a_0).*(1-invLogit(a+a_0)))))+rho*L^2
    return -1*hess
end;





function newton(y_i,a_0,L,rho,b,u)
    a = zeros(length(y_i),1)
    a_old = a
    iters = 0
    diff = 1.0
    while(diff >STOP_DIFF && iters< MAX_ITER )
        grad = gradient2(a_old,a_0,u,L,rho,b,y_i)
        hess = hessian(a_old,a_0, rho,L)
        a = a_old - pinv(hess)*grad
        diff = norm(a-a_old)
        a_old = a
        iters = iters+1
    end
    return a
end

newton (generic function with 1 method)

In [3]:


levels = 10
g = BinaryTree(levels)
n = nv(g)
b = (rand(n) .< 8 / n)*1. ;
g = randEdgeGen(g,1000)
A = Array{Int64,2}[]
L =  SparseMatrixCSC{Int64,Int64}[]
numnewnodes = 5
for i in 1:numnewnodes
    g = addPrefNode(g,b)
    push!(L,laplacian_matrix(g))
    connects = zeros(2^levels-1+i,1)  #-1 for -1 1 coding
    connects[neighbors(g,nv(g))] = 1
    push!(A,connects)
end

t = 2^levels-1+numnewnodes
t_0 = 2^levels-1
u = zeros(t,1)+0.2;
rho = 1.1
lambda = 1.1
a_0 = -7

-7

In [4]:
a = zeros(t,numnewnodes)
b = zeros(t)
u = zeros(t,numnewnodes)
#	alpha = 1.5  #relaxation parameter
iters = 0
diff = 1.0
b_old = b;

In [16]:
a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,b[1:length(A[i])],u[1:length(A[i]),i])


1025×1 Array{Float64,2}:
 0.0662768
 0.0673546
 0.0669865
 0.0649485
 0.0652506
 0.0671763
 0.0647635
 0.0627811
 0.0655064
 0.0678055
 0.0641931
 0.0784358
 0.063489 
 ⋮        
 0.0604062
 0.0661535
 0.0648501
 0.0672502
 0.0598653
 0.0682017
 0.062589 
 0.060786 
 0.0566185
 0.0542359
 0.0625448
 0.39449  

In [17]:
u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-b[1:length(A[i])])

1025-element Array{Float64,1}:
 -0.0019663  
 -0.00145963 
 -0.00149955 
 -0.00220314 
 -0.00190791 
 -0.00096844 
 -0.00217936 
 -0.00273334 
 -0.00211586 
 -0.000453873
 -0.00238917 
  0.00497202 
 -0.00247532 
  ⋮          
 -0.00329048 
 -0.00181626 
 -0.00211115 
 -0.00149675 
 -0.0036843  
 -0.00103988 
 -0.00280297 
 -0.00329401 
 -0.00421634 
 -0.00470552 
 -0.002986   
  0.209458   

In [12]:
sum(u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-b[1:length(A[i])]))

0.0

In [10]:
L[i]*a[:,i]

LoadError: DimensionMismatch("")

In [18]:
methods(newton)

# 1 method for generic function "newton":
newton(y_i, a_0, L, rho, b) at In[2]:25

In [28]:
function ADMM(A,L,t,t_0,new)
	a = zeros(t,new)
	b = zeros(t)
	u = zeros(t,new)
#	alpha = 1.5  #relaxation parameter
	iters = 0
	diff = 1.0
	b_old = b
	while(diff >STOP_DIFF && iters< MAX_ITER )
		#a update
		for i in t_0:t
			a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,b[1:length(A[i])],u[:,i])
			u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[:,i]-b[1:length(A[i])])
		end
		c = zeros(t)
		for i in t_0:t
			c = c+ u[:,i]'+rho(L[i]*a[:,i])/((t-t_0)*rho/2)
		end
		b = sign(c).*max(abs(c)-lambda/2,0) #soft(c)
		diff  = norm(b-b_old)
		b_old = b
	end
	return b
end


ADMM (generic function with 1 method)